In [ ]:
# import tensorflow
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM

In [ ]:
from src.db.db_utils import db_utils
from src.db.controller import init_async_db
from src.config import ETHER_protocols

transactions, blocks, future = await init_async_db(True)
db_utils.set_tables(transactions, blocks, future)

transaction_table = db_utils.get_transactions()


In [ ]:
protocol = ETHER_protocols.get('Uniswap').lower()
print(protocol)

In [ ]:
transactions_rows = await transaction_table.get_all_rows_by_criteria({'contract': protocol})
print(len(transactions_rows))
transactions_rows = transactions_rows[:-500]

In [ ]:
type(transactions_rows[0])

In [ ]:
import pandas as pd

df = pd.DataFrame([t.__dict__ for t in transactions_rows])

In [ ]:
df.head()

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [ ]:
# df.head(

In [ ]:
import plotly.express as px

#
#
fig = px.line(df.reset_index(), x='timestamp', y='gas_price', title='Gas Price')
fig.update_xaxes(
    rangeslider_visible = True
)
# fig.show()
# fig = px.line(df.reset_index(), x='timestamp', y='gas', title='Gas')
# fig.update_xaxes(
#     rangeslider_visible = True
# )
# fig.show()
# fig = px.line(df.reset_index(), x='timestamp', y='gas_percentage', title='Gas Relative to Total Gas')
# fig.update_xaxes(
#     rangeslider_visible = True
# )
# fig.show()
fig = px.line(df.reset_index(), x='timestamp', y='priority_fee', title='Priority Fee')
fig.update_xaxes(
    rangeslider_visible=True
)
fig.show()

In [ ]:
df = df.drop(columns=['_sa_instance_state', 'timestamp', 'id', 'gas', 'gas_percentage', 'contract', 'tx', 'gas_price',
                      'gas_percentage_by_limit_in_block'])
columns_titles = ["priority_fee"]
df = df.reindex(columns=columns_titles)
import numpy as np



def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

df = clean_dataset(df)

result = {}

# result['block'] = df.block.values
result['priority_fee'] = df.priority_fee.values



In [ ]:
df.head()

In [ ]:

np.any(np.isnan(df))

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


df = df.reset_index()
outliers_fraction=0.05
# scaler = MinMaxScaler((0, 1))
# df[['priority_fee']] = scaler.fit_transform(df[['priority_fee']])
# print(max(df.priority_fee.values))

data = df.to_numpy()
print(data)

In [ ]:
from tods.detection_algorithm.DeepLog import DeepLog
# from tods.detection_algorithm.DeepLog import

# params = Params()

transformer_DL = DeepLog(epochs=10)
transformer_DL.fit(data)

In [ ]:
prediction_labels_DL = transformer_DL.predict(data)
prediction_score_DL = transformer_DL.predict_score(data)
print("Prediction Labels\n", prediction_labels_DL)
print("Prediction Score\n", prediction_score_DL)

In [ ]:

result['anomaly_DeepLog'] = pd.Series(prediction_score_DL.flatten())
result['anomaly_DeepLog'] = result['anomaly_DeepLog'].apply(lambda x: x > 20)
result['anomaly_DeepLog'] = result['anomaly_DeepLog'].astype(int)
result['anomaly_DeepLog'].value_counts()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10,6))

#anomaly
a = df.loc[result['anomaly_DeepLog'] == 1]
outlier_index=list(a.index)
ax.plot(result['priority_fee'], color='black', label = 'Normal', linewidth=1.5)
ax.scatter(a.index ,a['priority_fee'], color='red', label = 'Anomaly', s=16)
ax.plot(pd.Series(prediction_score_DL.flatten()*10), color='blue', label = 'Score', linewidth=0.5)


plt.legend()
plt.title("Anamoly Detection Using DeepLog")
plt.xlabel('Date')
plt.ylabel('Transactions')
plt.show();

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# normalize the dataset
# scaler = MinMaxScaler(feature_range=(0, 1))
# df[['priority_fee']] = scaler.fit_transform(df[['priority_fee']])

In [ ]:
df = df.set_index('timestamp').resample('H').max()

In [ ]:
# from adtk.detector import SeasonalAD
# from adtk.visualization import plot
#
#
# seasonal_ad = SeasonalAD()
# anomalies = seasonal_ad.fit_detect(df)
# plot(df, anomaly=anomalies, anomaly_color="red", anomaly_tag="marker")

In [ ]:
X_train = df.timestamp
Y_train = df.priority_fee

from pyod.models.ecod import ECOD

clf = ECOD()
clf.fit((X_train, Y_train))

# get outlier scores
y_train_scores = clf.decision_scores_  # raw outlier scores on the train data
# y_test_scores = clf.decision_function(X_test)  # predict raw outlier scores on test

In [ ]:
from pyod.utils.example import visualize

y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# visualize the results
visualize('ECOD', X_train, Y_train, y_train_pred=y_train_pred,
          show_figure=True, save_figure=False)

In [ ]:
print(y_train_scores)
y_pred = clf.predict(df['timestamp'])

In [ ]:
# df = df.set_index('timestamp')

In [ ]:
fig = px.line(pred, title='Priority Fee')
fig.update_xaxes(
    rangeslider_visible=True
)
fig.show()